# Welcome to SMS Badwords Checker

In [2]:
# IMPORT DATA PACKAGE 
import pandas as pd 
import numpy as np
import pygsheets 
import string

In [3]:
# IMPORT DATA
gc = pygsheets.authorize(service_account_file="cobra-reader2-a20820469446.json")
lanina_url = gc.open_by_url('https://docs.google.com/spreadsheets/d/1obszkCQoE0ELOR1O0CrLVETUEmEIWlGuyAmK3FgWSJg/edit#gid=1060654066')
lanina_sheet =  lanina_url.worksheet('title','La Nina (Current)')
lanina = lanina_sheet.get_as_df()
badwords = gc.open_by_url('https://docs.google.com/spreadsheets/d/13AlQzsLpuUphMALUEgmlze2vu1oh4Fvv-4wFS1lu18I/edit#gid=0') 
badwords_df  = badwords[0].get_as_df()

/opt/anaconda3/lib/python3.8/site-packages/pygsheets/worksheet.py:1477: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')


In [61]:
financial_badwords = badwords_df['All SC Badwords'].tolist() + badwords_df['[ BRAND TERMS ]'].tolist() + badwords_df['[ PROFANITY ]'].tolist() + badwords_df['[ DRUG TERMS ]'].tolist()
non_fiancial_badwords = financial_badwords + badwords_df['Non-Financial SC Badwords'].tolist()
# remove '' from list 
non_fiancial_badwords = [str(x).lower() for x in non_fiancial_badwords if x != '']
financial_badwords = [str(x).lower() for x in financial_badwords if x != '']

# single offer check 
def check_single_content(content,sc):
    """ 
    Checks for badwords in the content of the ad. 
    Returns a dataframe with the badword and the number of times it appears in the ad. 
    """
    global Non_financial_sc
    global Financial_sc
    badwords = ''
    badwords_lyst = []
    Non_financial_sc = [str(x).lower() for x in Non_financial_sc if x != '']
    Financial_sc = [str(x).lower() for x in Financial_sc if x != '']

    if sc.lower() in Non_financial_sc:
        badwords = non_fiancial_badwords
    if sc.lower()  in Financial_sc:
        badwords = financial_badwords
    # check whether the content contains any badwords 
    shortcode = Non_financial_sc + Financial_sc
    shortcode = [str(x).lower() for x in shortcode if x != '']
    
    if content[0].lower() not in shortcode:
        return 'No shortcode found' 
    for i in content: 
        if str(i) in badwords:
            badwords_lyst.append(i)
    if len(badwords_lyst) > 0:
        return ','.join(badwords_lyst)
    else:
        return ''


def replace_punctuation(s):
    translator = str.maketrans('', '', string.punctuation)
    return s.translate(translator)

def check_content_final(content):
    content = content.lower() 
    sc = content[:3] 
    content = replace_punctuation(content).split(' ')
    result = check_single_content(content,sc)

    if result == 'No shortcode found':
        print("No Shortcode found. Please check your shortcode.") 
    elif result != '':
        print("We find badwords:",result)
    else:
        print("We don't find any badwords. You're good to go!")

# FOR SMS DMA TEAM

In [62]:
# check all the content from Lanina 
Financial_sc = ['FLC','DSS','SVT']
Non_financial_sc = ['MBC','HZB','UAA']
lanina['Content_words'] = lanina['Content'].str.lower().str.replace('[,.;@#?!&${}:]+','',regex= True).str.split(' ')
lanina = lanina[(lanina['Reporting Content ID'].isna() == False) & (lanina['Reporting Content ID']!='')]
lanina['badwords'] = lanina.apply(lambda row: check_single_content(row['Content_words'],row['Type']),axis=1)
Badwords_df = lanina.loc[(lanina['badwords'] != '') & (~lanina['Content Approval Status'].str.contains('Paused|Not Approved|Failed Testing', na = False)),['Reporting Content ID','Content Approval Status','Type','badwords','Content']]
Badwords_df

,Reporting Content ID,Content Approval Status,Type,badwords,Content
561,EE.12231.SC.FLC.451401,Approved - Pipeline Testing,FLC,guaranteed,FLC: You received a card offer. For guaranteed...
562,EE.12231.SC.MBC.451401,Approved - Pipeline Testing,MBC,guaranteed,MBC: Congrats! You have guaranteed approval fo...
600,CGC.11646.SC.HZB.451492,Approved - Passed,HZB,guaranteed,HZB: You have guaranteed approval! View your n...
601,CGC.11646.SC.MBC.451491,Approved - Passed,MBC,guaranteed,MBC: Guaranteed approval? See your new card te...
761,CGC.11646.SC.FLC.451631,Approved - Passed,FLC,guaranteed,"FLC: You received a $1,000 card offer. The twi..."
764,CGC.11646.SC.HZB.451632,Approved - Passed,HZB,guaranteed,HZB: You’ve earned it! View your guaranteed Go...
766,CGC.11646.SC.MBC.451632,Approved - Passed,MBC,guaranteed,MBC: You received (1) new card offer to your i...
899,SLF.12145.SC.HZB.451811,Pending - Content Split Testing,HZB,earnings,HZB: If you’re a {{coalesce|region|US}} reside...
1073,VAST.6322.SC.HZB.451972,Approved - Content Split Testing,HZB,guaranteed,HZB: Reward yourself with the benefits of a ne...
1074,VAST.6322.SC.MBC.451971,Approved - Content Split Testing,MBC,guaranteed,MBC: This platinum card offer comes with guara...


# For Content Team:

In [70]:
# enter content and the sc 
Financial_sc = ['FLC','DSS','SVT']
Non_financial_sc = ['MBC','HZB','UAA']
content = "MBC: It’s guaranteed to pick a lender for your e-deposit request, {{coalesce|First_name|partner}}!Request up to $40k: {{offer_target_url}}STOP to end"
check_content_final(content)



We find badwords: guaranteed,40k
